# Binary evolutions for the progenitor of J08408

Study of grid of binaries with a NS orbiting around a massive star. The aim of this notebook is to present a clear picture on the parameters which are related to J08408

## Configuration

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.ticker import MultipleLocator
from scipy.optimize import bisect
import pandas as pd
from scipy import interpolate

from utils import *

# some constants
one_third = 1e0 / 3e0
pi = 3.1415926535897932384626433832795028841971693993751e0
standard_cgrav = 6.67428e-8
Msun = 1.9892e33
Rsun = 6.9598e10
Hubble_time = 13.461701658024014e0  # Gyr
clight = 2.99792458e10
secyer = 3.1558149984e7

# ---------------------------------------------
# Set some global options
np.seterr(all="ignore")

# ---------------------------------------------
# Load data
root = ".."
run_location = f"{root}/data/natal-kicks"
mesa_run_location = f"{root}/data"
config_location = f"{root}/config"

iddist, wdist, thetadist, phidist = np.loadtxt(
    f"{run_location}/kicks-distribution.data", skiprows=1, unpack=True
)
idgrid, Pgrid, agrid, egrid, probgrid = np.loadtxt(
    f"{run_location}/target_orbits.data", skiprows=9, unpack=True
)

# load summary of mesa simulations
summary_of_progenitors = pd.read_csv(
    f"{mesa_run_location}/summary_of_star_plus_ns.data",
    delim_whitespace=True,
    skiprows=3,
)
additional_info_on_progenitors = pd.read_csv(
    f"{mesa_run_location}/additional_info_of_star_plus_ns.data", delim_whitespace=True
)
detail_progenitors = pd.read_csv(
    f"{mesa_run_location}/details_of_progenitors_of_j08408.data", delim_whitespace=True
)

mpl_style = f"{config_location}/style.mpl"
paper_style = f"{config_location}/paper-style.mpl"

## Parameters at core collapse

In [ ]:
m1precc = 8.3525627823294126e000
m2precc = 3.2637938645139933e001
aprecc = 7.3604203503769526e001
pprecc = 1.1424984631360623e001
r1precc = 2.4346822488037647e000
r2precc = 1.8891193348004055e001
remnantmass = 1.6601196874643882e000

## Kepler law & some auxiliary functions

In [ ]:
def a_to_P(separation, m1, m2):
    """Kepler law to go from separation to orbital period

    Parameters
    ----------
    a: binary separation in [Rsun]
    m1: mass of primary star in [Msun]
    m2: mass of secondary star in [Msun]

    Returns
    -------
    period: orbital period in [days]
    """
    separation = separation * Rsun  # in cm
    m1 = m1 * Msun
    m2 = m2 * Msun  # in g

    period = (
        2 * np.pi * np.sqrt(separation**3 / (standard_cgrav * (m1 + m2))) / (86400.0)
    )
    return period


def P_to_a(period, m1, m2):
    """
    Binary separation from known period

    Parameters
    ----------
    P: binary period in [days]
    M1: mass of primary star in [Msun]
    M2: mass of secondary star in [Msun]

    Returns
    -------
    a: binary separation in [Rsun]
    """
    period = period * 24e0 * 3600e0  # in sec
    m1 = m1 * Msun
    m2 = m2 * Msun  # in g
    tmp = standard_cgrav * (m1 + m2) * (period / (2 * np.pi)) ** 2
    separation = np.power(tmp, one_third)
    return separation / Rsun


def rlobe(m1, m2, separation):
    """Roche lobe approximation for a star using Eggleton (1983)
    formula

    Parameters
    ----------
    m1: mass of primary star in [Msun]
    m2: mass of secondary star in [Msun]
    separation: binary separation in [Rsun]

    Returns
    -------
    RL: Roche lobe of primary star in [Rsun]
    """
    one_third = 1e0 / 3e0
    q = np.power(m1 / m2, one_third)
    RL = 0.49e0 * q**2 / (0.6 * q**2 + np.log(1 + q))
    RL = separation * RL

    return RL


def get_line_of_constant_w(w):
    """For a given kick velocity, get orbital parameters matching velocity with a different
    set of theta values (angle in the orbital plane)

    Parameters
    ----------
    w : strength of the kick in [km s-1]

    Returns
    -------
    period : orbital period in [days]
    eccentricity : eccentricity of the binary
    """
    # get period & eccentricity from linear momentum conservation
    _, P_f, e_f, _, _, _, _, _, _ = binary_orbits_after_kick(
        aprecc, m1precc, m2precc, remnantmass, v, thetadist, 0, iddist
    )

    # interpolate (this do not work for fixed theta values, just for kick strength)
    f = interpolate.interp1d(P_f, e_f)

    # now use the interpolation to get lines of constant w
    xmin = np.min(P_f)
    xmax = np.max(P_f)
    xnew = np.logspace(np.log10(xmin), np.log10(xmax - 1), 1000)
    return xnew, f(xnew)


def get_points_of_constant_theta(theta):
    """For a given theta angle, get orbital parameters matching angle with a different
    set of kick velocity values

    Parameters
    ----------
    theta : angle of the kick in the orbital plane in the interval [0, 2*pi)

    Returns
    -------
    period : orbital period in [days]
    eccentricity : eccentricity of the binary
    """
    # get period & eccentricity from linear momentum conservation
    _, P_f, e_f, _, _, _, _, _, _ = binary_orbits_after_kick(
        aprecc, m1precc, m2precc, remnantmass, wdist, theta, 0, iddist
    )

    return P_f, e_f

def get_lines_of_constant_RRL(R_div_RL):
    """Line of constant ratio R/RL of a star in a binary
    
    Parameters
    ----------
    
    Returns
    -------
    """
    # constant lines of RLOF
    ecc_max = 0.999
    ecs = np.linspace(0, ecc_max)
    a_for_R_div_RL = np.zeros(len(ecs))
    a = np.logspace(-1,4)
    for i,e in enumerate(ecs):
        f = lambda a: r2precc / rlobe(m2precc, remnantmass, a * (1 - e)) - R_div_RL
        a_for_R_div_RL[i] = bisect(f, a=0, b=1e6, maxiter=100)
    
    mask = a_for_R_div_RL > m2precc / (1 + ecs)
    return a_to_P(a_for_R_div_RL[mask], m2precc, remnantmass), ecs[mask]

---

### Basic summary

In [ ]:
codes = list(set(summary_of_progenitors["termination_code_pm"]))
for code in codes:
    mask = summary_of_progenitors["termination_code_pm"] == code
    print("{}:".format(code), "n-simulations =", len(summary_of_progenitors[mask]))

## Details of how simulations ended

In [ ]:
plt.style.use("../config/style.mpl")
fig, ax = plt.subplots(figsize=(2.7, 2.7))
ax.set_xscale("log")

# axis labels
ax.set_xlabel("$P_{\\rm orb, post-CC}$ [days]")
ax.set_ylabel("$e_{\\rm post-CC}$")

# theoretical limits
ecc_max = 0.999
ecs = np.linspace(0, ecc_max)
ax.plot(
    a_to_P(aprecc / (1 + ecs), m2precc, remnantmass),
    ecs,
    ls="-",
    c="black",
    zorder=99,
)
ax.plot(
    a_to_P(aprecc / (1 - ecs), m2precc, remnantmass),
    ecs,
    ls="-",
    c="black",
    zorder=99,
)

# MESA runs
end_colors = ["firebrick", "steelblue"]  # , 'firebrick']
codes_dict = {
    "ce_merge": "CE phase",
    "high_atm_mdot": "$\\dot{M}_{\\rm RLOF} > 0.1$ M$_\odot$ yr$^{-1}$",
}
for k, code in enumerate(codes_dict.keys()):
    mask = (summary_of_progenitors["termination_code_pm"] == code) & \
    (summary_of_progenitors["num_models_pm"] > 1)
    seps_cc = summary_of_progenitors["porb_pm"][mask]
    eccs_cc = summary_of_progenitors["e_pm"][mask]
    ax.scatter(
        seps_cc, eccs_cc, s=3.1, marker="s", c=end_colors[k], label="{}".format(code),
        zorder=-1
    )
    
    mask = (summary_of_progenitors["termination_code_pm"] == code) & \
    (summary_of_progenitors["num_models_pm"] == 1)
    seps_cc = summary_of_progenitors["porb_pm"][mask]
    eccs_cc = summary_of_progenitors["e_pm"][mask]
    ax.scatter(
        seps_cc, eccs_cc, s=3.1, marker="s",
        c="gray",
        label="{}".format(code),
        zorder=-1
    )
        

# text on conditions at CC
ax.annotate(
    "Conditions at CC\n{} = {:.2f} {}\n{} = {:.2f} {} {} {} = {:.2f} {}\n{} = {:.2f} {}".format(
        "$a$",
        aprecc,
        "R$_\\odot$",
        "$m_1$",
        m1precc,
        "M$_\\odot$",
        "$\\rightarrow$",
        "$m_{\\rm NS}$",
        remnantmass,
        "M$_\\odot$",
        "$m_2$",
        m2precc,
        "M$_\\odot$",
    ),
    fontsize=6,
    xy=(0.48, 0.30),
    xycoords="figure fraction",
)

# axis limits & ticks
ax.set_xlim(4, 1.9e3)
ax.set_yticks(np.arange(0, 1.1, 0.2))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))
ax.set_ylim(0,1)
ax.tick_params('y', reset=True, zorder=99)

# legends
f = lambda c, s: plt.plot([], [], ls="none", marker="s", color=c, markersize=s)[0]
handles = [f(color, 4) for color in end_colors]
handles.append((f("gray", 4)))
legends = ["{}".format(codes_dict[event]) for event in codes_dict.keys()]
legends.append(("init MT"))
fig.legend(handles, legends, loc="center", ncol=3, bbox_to_anchor=(0.51, 0.91));

### Constant lines of R/RL

In [ ]:
plt.style.use("../config/style.mpl")
fig, ax = plt.subplots(figsize=(2.7, 2.7))
ax.set_xscale("log")

# axis labels
ax.set_xlabel("$P_{\\rm orb, post-CC}$ [days]")
ax.set_ylabel("$e_{\\rm post-CC}$")

# theoretical limits
ecc_max = 0.999
ecs = np.linspace(0, ecc_max)
ax.plot(
    a_to_P(aprecc / (1 + ecs), m2precc, remnantmass),
    ecs,
    ls="-",
    c="black",
    zorder=99,
)
ax.plot(
    a_to_P(aprecc / (1 - ecs), m2precc, remnantmass),
    ecs,
    ls="-",
    c="black",
    zorder=99,
)

# MESA runs
end_colors = ["firebrick", "steelblue"]  # , 'firebrick']
codes_dict = {
    "ce_merge": "CE phase",
    "high_atm_mdot": "$\\dot{M}_{\\rm RLOF} > 0.1$ M$_\odot$ yr$^{-1}$",
}
for k, code in enumerate(codes_dict.keys()):
    mask = (summary_of_progenitors["termination_code_pm"] == code) & (
        summary_of_progenitors["num_models_pm"] > 1
    )
    seps_cc = summary_of_progenitors["porb_pm"][mask]
    eccs_cc = summary_of_progenitors["e_pm"][mask]
    ax.scatter(
        seps_cc,
        eccs_cc,
        s=3.1,
        marker="s",
        c=end_colors[k],
        label="{}".format(code),
        zorder=-1,
    )

    mask = (summary_of_progenitors["termination_code_pm"] == code) & (
        summary_of_progenitors["num_models_pm"] == 1
    )
    seps_cc = summary_of_progenitors["porb_pm"][mask]
    eccs_cc = summary_of_progenitors["e_pm"][mask]
    ax.scatter(
        seps_cc,
        eccs_cc,
        s=3.1,
        marker="s",
        c="gray",
        label="{}".format(code),
        zorder=-1,
    )

# text on conditions at CC
ax.annotate(
    "Conditions at CC\n{} = {:.2f} {}\n{} = {:.2f} {} {} {} = {:.2f} {}\n{} = {:.2f} {}".format(
        "$a$",
        aprecc,
        "R$_\\odot$",
        "$m_1$",
        m1precc,
        "M$_\\odot$",
        "$\\rightarrow$",
        "$m_{\\rm NS}$",
        remnantmass,
        "M$_\\odot$",
        "$m_2$",
        m2precc,
        "M$_\\odot$",
    ),
    fontsize=6,
    xy=(0.48, 0.30),
    xycoords="figure fraction",
)

for R_div_RL in [1.0, 1.5]:
    ptmp, etmp = get_lines_of_constant_RRL(R_div_RL)
    ax.plot(ptmp, etmp, color="black", ls="--", zorder=0)

# axis limits & ticks
ax.set_xlim(4, 1.9e3)
ax.set_yticks(np.arange(0, 1.1, 0.2))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))
ax.set_ylim(0, 1)
ax.tick_params("y", reset=True, zorder=99)

# a = a_pre / (1 - e)
# => e = -1 + (a_pre / a)
porbs = np.logspace(np.log10(np.min(Pgrid)), np.log10(np.max(Pgrid)))
ecss = np.zeros(len(porbs))
for k, p in enumerate(porbs):
    a = P_to_a(p, m2precc, remnantmass)
    ecss[k] = -1 + aprecc / a
ax.fill_between([4.0, 7], 0.01, 0.41, color="white", zorder=1)
ax.fill_between([4.0, 6], 0.01, 0.58, color="white", zorder=1)

# legends
# legends
f = lambda c, s: plt.plot([], [], ls="none", marker="s", color=c, markersize=s)[0]
handles = [f(color, 4) for color in end_colors]
handles.append((f("gray", 4)))
legends = ["{}".format(codes_dict[event]) for event in codes_dict.keys()]
legends.append(("init MT"))
fig.legend(handles, legends, loc="center", ncol=3, bbox_to_anchor=(0.51, 0.91))

ax.annotate(
    "$R/R_{} = {}$".format("{\\rm RL}", 1.0),
    xy=(8, 0.54),
    zorder=99,
    rotation=55,
    fontsize=6,
    # bbox=dict(facecolor="lightgray", edgecolor="dimgray", boxstyle="round, pad=0.2"),
);

ax.annotate(
    "$R/R_{} = {}$".format("{\\rm RL}", 1.5),
    xy=(6, 0.64),
    zorder=99,
    rotation=50,
    fontsize=6,
    # bbox=dict(facecolor="lightgray", edgecolor="dimgray", boxstyle="round, pad=0.2"),
);

## Eccentricity & orbital periods

In [ ]:
plt.style.use("../config/style.mpl")
fig, ax = plt.subplots(figsize=(2.9, 2.9))
ax.set_xscale("log");

# axis labels
ax.set_xlabel("$P_{\\rm orb, post-CC}$ [days]")
ax.set_ylabel("$e_{\\rm post-CC}$");

# axis limits
ax.set_xlim(4, 1.9e3)
ax.set_ylim(0, 1);

# theoretical limits
ecc_max = 0.999
ecs = np.linspace(0, ecc_max)
ax.plot(
    a_to_P(aprecc / (1 + ecs), m2precc, remnantmass),
    ecs,
    ls="-",
    c="black",
    zorder=99,
)
ax.plot(
    a_to_P(aprecc / (1 - ecs), m2precc, remnantmass),
    ecs,
    ls="-",
    c="black",
    zorder=99,
);

# colormap settings
bounds = np.arange(0, 5, 1)
norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)
cm = plt.get_cmap("RdYlBu");

mask = (additional_info_on_progenitors["MT_case"] == "none") & \
(additional_info_on_progenitors["number_of_models"] > 1)
x = additional_info_on_progenitors["porb"][mask]
y = additional_info_on_progenitors["e"][mask]
z = additional_info_on_progenitors["min_chi2"][mask]
ax.scatter(x, y, s=1.5, c=z, cmap=cm, norm=norm, alpha=0.9, marker="s")

mask = (additional_info_on_progenitors["MT_case"] != "none") & \
(additional_info_on_progenitors["number_of_models"] > 1)
x = additional_info_on_progenitors["porb"][mask]
y = additional_info_on_progenitors["e"][mask]
z = additional_info_on_progenitors["min_chi2"][mask]
cb = ax.scatter(x, y, s=1.5, c=z, cmap=cm, norm=norm, alpha=0.9, marker="s")
cbar = fig.colorbar(cb, ax=ax, label="$\\min(\\chi^2)$", extend="max")
cbar.minorticks_off();

In [ ]:
additional_info_on_progenitors.keys()

In [ ]:
detail_progenitors.keys()

## X-ray binary phase timescale

In [ ]:
plt.style.use('../config/style.mpl')
fig, ax = plt.subplots(figsize=(2.7,2.7))
ax.set_xscale('log');

# axis labels
latex = '{\\rm orb}'
ax.set_xlabel(f'$P_{latex}$ [days]')
ax.set_ylabel('eccentricity');

# axis limits
ax.set_xlim(3, 2e3)
ax.set_ylim(0.49, 0.89);

# colormap settings
# bounds = np.arange(0,6e5,1000)
# norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)
# cm = plt.get_cmap('RdYlBu')

mask = additional_info_on_progenitors['t_xrb'] < 100
x = additional_info_on_progenitors['porb'][mask]
y = additional_info_on_progenitors['e'][mask]
z = additional_info_on_progenitors['min_chi2'][mask]
ax.scatter(x, y, s=0.1, c='gray', norm=norm, alpha=0.9, marker='o')

mask = additional_info_on_progenitors['t_xrb'] > 100
x = additional_info_on_progenitors['porb'][mask]
y = additional_info_on_progenitors['e'][mask]
z = additional_info_on_progenitors['t_xrb'][mask]
cb = ax.scatter(x, y, s=1, c=z, vmin=100, vmax=2000)# , cmap=cm, norm=norm, alpha=0.9, marker='o');

cbar = fig.colorbar(cb, ax=ax, label='$t_{\\rm XRB} [yrs]$', extend='max')
cbar.minorticks_off();